In [80]:
import pandas as pd
import sqlite3

In [19]:
# #import the library sqlite3
# import sqlite3

# #create a new database and open a database connection with the name example
# example = sqlite3.connect("example.db")

# # create a data cursor cur in order execute SQL statements and fetch results from SQL queries
# cur = example.cursor()

# # Create table with the column date, side, symbol, qty, price
# cur.execute("CREATE TABLE stock (date VarChar(10), side VarChar(20), symbol VarChar(20), qty float, price float)")

# # Insert 3 row of data: the stock APPLE ( AAPL) with the date of transaction, the nature of side ( BUY or SELL),
# #the quantity of the stock traded and the price
# cur.execute("""
#     INSERT INTO stock (date, side, symbol, qty, price) VALUES
#         ('2021-01-01', 'BUY', 'AAPL', 100, 148.48),
#         ('2021-01-02', 'SELL', 'AAPL', 100, 149.11),
#         ('2021-01-03', 'BUY', 'AAPL', 75, 144.27),
#         ('2021-01-04', 'SELL', 'AAPL', '75', '146.39')
#         """) 

# # Save (commit) the changes
# example.commit() 

# #return all resulting rows
# res = cur.execute("SELECT * FROM stock")
# res.fetchall()

# # Close the connection
# example.close()

# # #for more informations : https://docs.python.org/3/library/sqlite3.html

## the construction of a financial database

Let’s start simple and only consider the structure for the equity data that we are gathering for now – in our case the constituents of the S&P500 index. We can tag on the various tables and possibly whole databases to the mix as and when we need to, i.e. when we start to deal with the spot currency data and fund data that we plan to include.

the  plan is to segregate theequity data into 4 separate tables, using unique ids and foreign keys to create the relevant relationships and links between them.

The tables will be as follows:


<img src='Tables.png'>

<img src='C:\Users\User\Desktop\Formations_Data-LAPTOP-JP680GIQ\Formations_Data-LAPTOP-JP680GIQ\Organisme_formation\Ib_cegos\AXA\MCD.png'>

In [30]:
#import the library os and the module Error of the library sqlite3
import os
from sqlite3 import Error
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    #YOUR CODE 
    conn = None
    try: 
        conn = sqlite3.connect(db_file)
        print("connection établie")
        return conn
    except :
        print ("erreur")
        


def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    #YOUR CODE 
    try : 
        cur = conn.cursor()
        cur.execute(create_table_sql)  
              
    except :
        print("erreur")

#the name of the database
db_name = "data.db"
        
def main():

    
    # create the tables according to the MCD
    sql_create_exchange_table = """ CREATE TABLE IF NOT EXISTS exchange (
                                        id integer ,
                                        name text NOT NULL,
                                        currency VarChar(5),
                                        code text NOT NULL UNIQUE,
                                        PRIMARY KEY(id)
                                    ); """
    
    sql_create_company_table = """CREATE TABLE IF NOT EXISTS company (
                                    id integer ,
                                    name text NOT NULL,
                                    industry text,
                                    sector text,
                                    hq_location text,
                                    security_id integer,
                                    PRIMARY KEY(id),
                                    FOREIGN KEY (security_id) REFERENCES security (id)
                                );"""
    
    
    sql_create_security_table = """CREATE TABLE IF NOT EXISTS security (
                                id integer,
                                ticker text NOT NULL UNIQUE,
                                name text NOT NULL,
                                company_id integer,
                                exchange_id integer,
                                PRIMARY KEY(id)
                                FOREIGN KEY (company_id) REFERENCES company (id),
                                FOREIGN KEY (exchange_id) REFERENCES exchange (id)
                            );"""
    
    sql_create_security_price_table =  """CREATE TABLE IF NOT EXISTS security_price (
                        id integer,
                        date text NOT NULL,
                        open decimal NOT NULL,
                        high decimal NOT NULL,
                        low decimal NOT NULL,
                        close decimal NOT NULL,
                        volume integer,
                        adj_close decimal NOT NULL,
                        security_id integer,
                        PRIMARY KEY(id),
                        FOREIGN KEY (security_id) REFERENCES security (id)
                    );"""

    
    # create a database connection
    #YOUR CODE
    conn = create_connection(db_name)
    
    
    # create tables
    if conn is not None:
        # create exchange table
        #YOUR CODE
        create_table(conn, sql_create_exchange_table)
    
        # create company table
        #YOUR CODE
        create_table(conn, sql_create_company_table)
        
        # create security table
        #YOUR CODE
        create_table(conn, sql_create_security_table)
        
        # create security_price table
        #YOUR CODE
        create_table(conn, sql_create_security_price_table)
        
        
    else:
        print("Error! cannot create the database connection.")
main()

connection établie


In [21]:
#import the library pandas , requests and io
exchange_data = pd.read_csv("https://www.iso20022.org/sites/default/files/ISO10383_MIC/ISO10383_MIC.csv", encoding='iso-8859-1' )


In [22]:
exchange_data.head(3)

,MIC,OPERATING MIC,OPRT/SGMT,MARKET NAME-INSTITUTION DESCRIPTION,LEGAL ENTITY NAME,LEI,MARKET CATEGORY CODE,ACRONYM,ISO COUNTRY CODE (ISO 3166),CITY,WEBSITE,STATUS,CREATION DATE,LAST UPDATE DATE,LAST VALIDATION DATE,EXPIRY DATE,COMMENTS
0,DRSP,DRSP,OPRT,EURONEXT UK - REPORTING SERVICES,EURONEXT LONDON LIMITED,969500HMVSZ0TCV65D58,APPA,NaN,GB,LONDON,WWW.EURONEXT.COM,ACTIVE,20210927,20210927,20210927.0,NaN,APPROVED PUBLICATION ARRANGEMENT.
1,XCNQ,XCNQ,OPRT,CANADIAN SECURITIES EXCHANGE,"CNSX MARKETS, INC.",NaN,RMKT,CSE LISTED,CA,TORONTO,WWW.THECSE.COM,ACTIVE,20090427,20210927,20210927.0,NaN,FORMERLY KNOWN AS PURE (CSE OTHER LISTED) FORM...
2,PURE,XCNQ,SGMT,CANADIAN SECURITIES EXCHANGE - PURE,"CNSX MARKETS, INC.",NaN,NSPD,CSE-PURE,CA,TORONTO,WWW.THECSE.COM,ACTIVE,20061225,20210927,20210927.0,NaN,FORMERLY KNOWN AS PURE (CSE OTHER LISTED) FORM...


In [23]:
exchange_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2561 entries, 0 to 2560
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   MIC                                  2561 non-null   object 
 1   OPERATING MIC                        2561 non-null   object 
 2   OPRT/SGMT                            2561 non-null   object 
 3   MARKET NAME-INSTITUTION DESCRIPTION  2561 non-null   object 
 4   LEGAL ENTITY NAME                    495 non-null    object 
 5   LEI                                  1833 non-null   object 
 6   MARKET CATEGORY CODE                 2559 non-null   object 
 7   ACRONYM                              890 non-null    object 
 8   ISO COUNTRY CODE (ISO 3166)          2560 non-null   object 
 9   CITY                                 2559 non-null   object 
 10  WEBSITE                              2387 non-null   object 
 11  STATUS                        

In [24]:
exchange_data = exchange_data[['ISO COUNTRY CODE (ISO 3166)', 'MIC', 'MARKET NAME-INSTITUTION DESCRIPTION', 'ACRONYM']]
exchange_data.head()


,ISO COUNTRY CODE (ISO 3166),MIC,MARKET NAME-INSTITUTION DESCRIPTION,ACRONYM
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE
3,GB,ZODM,ZODIA MARKETS,NaN
4,US,FNFX,FENICS FX ECN,NaN


In [25]:
exchange_data.rename(columns={'ISO COUNTRY CODE (ISO 3166)' : 'country_code', 
                              'MIC':'code', 
                              'MARKET NAME-INSTITUTION DESCRIPTION': 'name',
                              'ACRONYM':'acronym'}, inplace=True)
exchange_data.head()

,country_code,code,name,acronym
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE
3,GB,ZODM,ZODIA MARKETS,NaN
4,US,FNFX,FENICS FX ECN,NaN


In [26]:
exchange_data['id'] = exchange_data.index
exchange_data.head()

,country_code,code,name,acronym,id
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN,0
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED,1
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE,2
3,GB,ZODM,ZODIA MARKETS,NaN,3
4,US,FNFX,FENICS FX ECN,NaN,4


In [27]:
exchange_data['country_code']

0       GB
1       CA
2       CA
3       GB
4       US
        ..
2556    US
2557    NO
2558    US
2559    US
2560    GB
Name: country_code, Length: 2561, dtype: object

In [28]:
# create the column currency in the the dataframe exhange_data whose values 
#are mapped with the country code : US => USD, GB =>GBP, DE =>EUR
mapping = {"US": "USD", "GB": "GBP", "DE": "EUR"}
exchange_data['currency'] = exchange_data['country_code'].map(mapping)
exchange_data[['id', 'name', 'currency', 'code']].head(30)

,id,name,currency,code
0,0,EURONEXT UK - REPORTING SERVICES,GBP,DRSP
1,1,CANADIAN SECURITIES EXCHANGE,NaN,XCNQ
2,2,CANADIAN SECURITIES EXCHANGE - PURE,NaN,PURE
3,3,ZODIA MARKETS,GBP,ZODM
4,4,FENICS FX ECN,USD,FNFX
5,5,NASDAQ OSLO ASA,NaN,NORX
6,6,PORTFOLIO STOCK EXCHANGE,NaN,POSE
7,7,PUNDION LLC,USD,PUND
8,8,UNICREDIT BULBANK AD,NaN,UCBG
9,9,ASX - NEW ZEALAND FUTURES & OPTIONS,NaN,NZFX


In [34]:
# save the data in a sql table exchange in the data db_name
#YOUR CODE
conn = sqlite3.connect(db_file)
exchange_data.to_sql('exchange_data', conn, if_exists='append', index = False)

2561

In [81]:
# scrape wiki table with symbols and details of Dow Jones constituents
dj_constituents = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average#Components')[1]
dj_constituents.rename(columns={'Company':'name', 'Industry': 'industry', 'Symbol': 'ticker'}, inplace=True)
dj_constituents.head(30)

,name,Exchange,ticker,industry,Date added,Notes,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,As Minnesota Mining and Manufacturing,2.88%
1,American Express,NYSE,AXP,Financial services,1982-08-30,NaN,3.56%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,NaN,4.88%
3,Apple,NASDAQ,AAPL,Information technology,2015-03-19,NaN,3.15%
4,Boeing,NYSE,BA,Aerospace and defense,1987-03-12,NaN,3.40%
5,Caterpillar,NYSE,CAT,Construction and Mining,1991-05-06,NaN,4.19%
6,Chevron,NYSE,CVX,Petroleum industry,2008-02-19,Also 1930-07-18 to 1999-11-01,3.05%
7,Cisco,NASDAQ,CSCO,Information technology,2009-06-08,NaN,1.00%
8,Coca-Cola,NYSE,KO,Drink industry,1987-03-12,Also 1932-05-26 to 1935-11-20,1.28%
9,Disney,NYSE,DIS,Broadcasting and entertainment,1991-05-06,NaN,2.32%


In [82]:
# scrape wiki table with symbols and details of s&P500 constituents
sp_constituents = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]#.to_csv('constintuents_current.csv', index=False)
sp_constituents.rename(columns={'Symbol': 'ticker', 'Security':'name', 'Headquarters Location': 'hq_location',
                                'GICS Sector': 'sector', 'GICS Sub-Industry':'industry'},inplace=True)
sp_constituents.head()

,ticker,name,SEC filings,sector,industry,hq_location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [94]:
#concetenate the previous tables/dataframe sp_constituents & dj_constituents with  the columns'name', 'industry', 'sector', 'hq_location'

#drop duplicate
#sort
#reset index
#create the colum id based on the index
#YOUR CODE 


company_table = pd.concat([dj_constituents[['name', 'industry']], 
                           sp_constituents[['name', 'sector','industry', 'hq_location']]])

#drop duplicate
company_table.drop_duplicates(inplace=True)
#sort
company_table.sort_values
#reset index
company_table.reset_index()
#create the colum id based on the index
company_table['id']=company_table.index


In [95]:
company_table

,name,industry,sector,hq_location,id
0,3M,Conglomerate,NaN,NaN,0
1,American Express,Financial services,NaN,NaN,1
2,Amgen,Biopharmaceutical,NaN,NaN,2
3,Apple,Information technology,NaN,NaN,3
4,Boeing,Aerospace and defense,NaN,NaN,4
...,...,...,...,...,...
498,Yum! Brands,Restaurants,Consumer Discretionary,"Louisville, Kentucky",498
499,Zebra Technologies,Electronic Equipment & Instruments,Information Technology,"Lincolnshire, Illinois",499
500,Zimmer Biomet,Health Care Equipment,Health Care,"Warsaw, Indiana",500
501,Zions Bancorporation,Regional Banks,Financials,"Salt Lake City, Utah",501


In [97]:

#concetenate the previous tables/dataframe sp_constituents & dj_constituents with  the columns'ticker', 'name'
#drop duplicate
#sort
#reset index
#create the colum id based on the index

#YOUR CODE 
security_table = pd.concat([dj_constituents[['name', 'ticker']], 
                           sp_constituents[['name', 'ticker']]])

#drop duplicate
security_table.drop_duplicates(inplace=True)
#sort
security_table.sort_values
#reset index
security_table.reset_index()
#create the colum id based on the index
security_table['id']=security_table.index

In [98]:
security_table

,name,ticker,id
0,3M,MMM,0
1,American Express,AXP,1
2,Amgen,AMGN,2
3,Apple,AAPL,3
4,Boeing,BA,4
...,...,...,...
498,Yum! Brands,YUM,498
499,Zebra Technologies,ZBRA,499
500,Zimmer Biomet,ZBH,500
501,Zions Bancorporation,ZION,501


In [106]:
#create the foreign key :  populating the security table with the company_id
#first step : create a dictionary with key, value pairs being the name of the company and the corresponding id 
#within its own table, which will be our foreign key in the second table.

#YOUT CODE 
company_id_mapper = dict(zip(company_table['name'], company_table['id']))

{'3M': 0,
 'American Express': 32,
 'Amgen': 39,
 'Apple': 3,
 'Boeing': 72,
 'Caterpillar': 5,
 'Chevron': 6,
 'Cisco': 112,
 'Coca-Cola': 8,
 'Disney': 152,
 'Dow': 10,
 'Goldman Sachs': 224,
 'Home Depot': 12,
 'Honeywell': 237,
 'IBM': 245,
 'Intel': 252,
 'Johnson & Johnson': 266,
 'JPMorgan Chase': 268,
 "McDonald's": 307,
 'Merck': 19,
 'Microsoft': 317,
 'Nike': 21,
 'Procter & Gamble': 381,
 'Salesforce': 409,
 'Travelers': 24,
 'UnitedHealth': 25,
 'Verizon': 470,
 'Visa': 27,
 'Walgreens Boots Alliance': 479,
 'Walmart': 480,
 'A. O. Smith': 1,
 'Abbott': 2,
 'AbbVie': 3,
 'Abiomed': 4,
 'Accenture': 5,
 'Activision Blizzard': 6,
 'ADM': 7,
 'Adobe Inc.': 8,
 'ADP': 9,
 'Advance Auto Parts': 10,
 'AES Corporation': 11,
 'Aflac': 12,
 'Agilent Technologies': 13,
 'Air Products and Chemicals': 14,
 'Akamai': 15,
 'Alaska Air Group': 16,
 'Albemarle Corporation': 17,
 'Alexandria Real Estate Equities': 18,
 'Align Technology': 19,
 'Allegion': 20,
 'Alliant Energy': 21,
 'Allst

In [109]:
##### Seconde step: create the foreign key company id 
#YOUR CODE 
security_id_mapper = dict(zip(security_table['name'], security_table['id']))
company_table['security_id']=company_table['name'].map(security_id_mapper)
company_table

In [112]:
#create the foreign key : populate the company table with the security_id:
#YOUR CODE 
security_table['compagny_id']=security_table['name'].map(company_id_mapper)
security_table

,name,ticker,id,compagny_id
0,3M,MMM,0,0
1,American Express,AXP,1,32
2,Amgen,AMGN,2,39
3,Apple,AAPL,3,3
4,Boeing,BA,4,72
...,...,...,...,...
498,Yum! Brands,YUM,498,498
499,Zebra Technologies,ZBRA,499,499
500,Zimmer Biomet,ZBH,500,500
501,Zions Bancorporation,ZION,501,501


In [120]:
#Reda the nyse.csv and the nasdaq.csv
#YOUR CODE 
nyse = pd.read_csv('nyse.csv')
nyse.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,DDD,3D Systems Corporation,7.66,$927.89M,NaN,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/ddd,NaN
1,MMM,3M Company,169.80,$97.81B,NaN,Health Care,Medical/Dental Instruments,https://old.nasdaq.com/symbol/mmm,NaN
2,WBAI,500.com Limited,2.95,$126.85M,2013.0,Consumer Services,Services-Misc. Amusement & Recreation,https://old.nasdaq.com/symbol/wbai,NaN
3,EGHT,8x8 Inc,16.75,$1.75B,NaN,Technology,EDP Services,https://old.nasdaq.com/symbol/eght,NaN
4,AHC,A.H. Belo Corporation,1.44,$34.39M,NaN,Consumer Services,Newspapers/Magazines,https://old.nasdaq.com/symbol/ahc,NaN


In [117]:
nasdaq = pd.read_csv('nasdaq.csv')
nasdaq.head()

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9
0,VCVC,10X Capital Venture Acquisition Corp,NaN,0.000000e+00,NaN,NaN,NaN,NaN,https://old.nasdaq.com/symbol/vcvc,NaN
1,VCVCU,10X Capital Venture Acquisition Corp,NaN,0.000000e+00,NaN,NaN,NaN,NaN,https://old.nasdaq.com/symbol/vcvcu,NaN
2,TXG,"10x Genomics, Inc.",142.27,1.520301e+10,NaN,2019.0,Capital Goods,Biotechnology: Laboratory Analytical Instruments,https://old.nasdaq.com/symbol/txg,NaN
3,YI,"111, Inc.",7.00,1.785821e+08,25511734.0,2018.0,Health Care,Medical/Nursing Services,https://old.nasdaq.com/symbol/yi,NaN
4,PIH,"1347 Property Insurance Holdings, Inc.",3.05,1.506076e+07,NaN,2014.0,Finance,Property-Casualty Insurers,https://old.nasdaq.com/symbol/pih,NaN


In [133]:
#populate the exchange_id column : assign the correct exchange_id for each stock in the security table:
# the exchange_id of the nyse stocks will be 1300
#YOUR CODE
a = 1300
for i in len(nyse['Name']):
    
    nyse['id']= i.append(1300)
# nyse_id_mapper = dict(zip(nyse['Name'], nyse['id']))
# security_table['exchange_id']=nyse['Name'].map(security_id_mapper)
# security_table
nyse['id']=1

TypeError: 'int' object is not iterable

In [ ]:
#populate the exchange_id column : assign the correct exchange_id for each stock in the security table:
# the exchange_id of  will be 1300 for the nyse stocks and 367 for the nasdaq stocks
#YOUR CODE

#YOUR CODE 
security_table.head(30)

In [ ]:
#save the data in the sql table security and company of the database db_name
#YOUR CODE 

In [ ]:
#Extraction the prices of YAHOO: WARNING this action may take 15 min !!!
from tqdm import tqdm_notebook as tqdm
import pandas_datareader.data as web
stock_pricing_dfs = []
for stock_id in tqdm(security_table['id']):
    try:
        stock_pricing_df = web.DataReader(security_table.iloc[stock_id]['ticker'], 
                           start='2010-1-1', 
                           end='2020-08-31', 
                           data_source='yahoo')
        stock_pricing_df['security_id'] = stock_id
        stock_pricing_dfs.append(stock_pricing_df)
    except:
        pass
security_price_table = pd.concat(stock_pricing_dfs)

In [ ]:
security_price_table.info()

In [ ]:
#insert data in the table security_price
security_price_table.columns = ['high', 'low', 'open', 'close', 'volume', 'adj_close', 'security_id']
security_price_table.reset_index(inplace=True)
security_price_table['id'] = security_price_table.index
print(security_price_table.head())
security_price_table.to_sql("security_price", conn, if_exists="append", index=False)